In [7]:
import requests
import json
import pandas as pd
from pandas import json_normalize
import config
from db_manager import DBManager
import os
from rest_api import Api
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from urllib.parse import quote_plus
from sqlalchemy import create_engine
import itertools
import logging

In [8]:
logging.basicConfig(
    format='%(levelname)s-%(lineno)s-%(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level='INFO'
)
logger=logging.getLogger(__name__)

#### Show all columns without truncating

Today's date

In [9]:
today = date.today()
logger.info(f'Current date: {today}')

INFO-__main__:2-Current date: 2023-08-31


Adding months to today's date. Near term travel dates 30 - 60 days ahead

In [10]:
departure_date_1_month = today + relativedelta(months=1)
departure_date_2_month = today + relativedelta(months=2)
return_date_1_month = departure_date_2_month
return_date_2_month = today + relativedelta(months=3)
logger.info(f'Departure_dates:{departure_date_1_month} and {departure_date_2_month} \nReturn dates:{return_date_1_month} and {return_date_2_month}')

INFO-__main__:5-Departure_dates:2023-09-30 and 2023-10-31 
Return dates:2023-10-31 and 2023-11-30


Get the nearby festival departure dates by start_date from the public.indian_holidays table

In [6]:
# credentials=config.pg_credentials
# db = DBManager(credentials=credentials)
# df_departure_arrival_dates = db.run_query(query_file_name='festival_dates')

In [7]:
#df_departure_arrival_dates 

Apply the combination of params to get data for all possible combinations of depart, arrival dates and depart arrival airports

Combination itertools usage. Im calculating the total combinations by multiplying the number of possibilities (unique values) for each variable<br />
Departure airport: 2 possibilities<br />
Arrival airport: 4 possibilities<br />
Departure date: 2 possibilities<br />
Arrival date: 1 possibilities<br />
Total combinations = 2 x 4 x 2 x 1 = 16 combinations

#### Travel payouts API to get flight search results for each combination in the list 

--------------------------------------------------------

API test for amadeus

POST request to the Amadeus authorization server to get the access token

In [11]:
response = requests.post(url=config.url_token, headers=config.headers_token, data=config.data)

if response.status_code == 200:
  # API call succeeded
  token = response.json()['access_token'] 
  logger.info('Successful access token generation')
else:
  # API call failed
  print(response.text)

INFO-__main__:6-Successful access token generation


In [12]:
#Combinations of destination and origin airport codes
origins = config.params['originLocationCode']
destinations = config.params['destinationLocationCode']
departure_dates=[departure_date_1_month,departure_date_2_month]
# return_dates=[return_date_1_month,return_date_2_month]

Creating combinations of Origin and Destination to be passed as parameters to the api call

trial

In [13]:
headers = {
  'client_id': config.client_id, 
  'client_secret': config.client_secret,
  'Authorization': f'Bearer {token}'
  }

url = config.url

response = []
counter = 0

for origin, dest, depart in itertools.product(origins, destinations, departure_dates):

    params={
        'originLocationCode': origin,
        'destinationLocationCode':dest, 
        'departureDate': depart, 
        'returnDate' : None,
        'adults':1,
        'children':None,
        'infants':None,
        'travelClass':None,
        'currencyCode':'EUR',
        'maxPrice' : None
        }


    #to be worked on later for by calling rest_api module
    # api = Api()
    # resp = api.make_flight_api_request(url,headers,params)
    resp = requests.get(url, headers=headers, params=params)
    counter = counter+1
    resp = resp.json()
    response.append(resp)
    logger.info(f'API call {counter} succeeded for flight offers')
    # else:
    #     # API call failed
    #     logger.info('API call failed')


INFO-__main__:35-API call 1 succeeded for flight offers
INFO-__main__:35-API call 2 succeeded for flight offers
INFO-__main__:35-API call 3 succeeded for flight offers
INFO-__main__:35-API call 4 succeeded for flight offers
INFO-__main__:35-API call 5 succeeded for flight offers
INFO-__main__:35-API call 6 succeeded for flight offers
INFO-__main__:35-API call 7 succeeded for flight offers
INFO-__main__:35-API call 8 succeeded for flight offers
INFO-__main__:35-API call 9 succeeded for flight offers
INFO-__main__:35-API call 10 succeeded for flight offers
INFO-__main__:35-API call 11 succeeded for flight offers
INFO-__main__:35-API call 12 succeeded for flight offers
INFO-__main__:35-API call 13 succeeded for flight offers
INFO-__main__:35-API call 14 succeeded for flight offers
INFO-__main__:35-API call 15 succeeded for flight offers
INFO-__main__:35-API call 16 succeeded for flight offers


In [30]:
print(response)

[{'meta': {'count': 123, 'links': {'self': 'https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=BER&destinationLocationCode=DEL&departureDate=2023-09-30&adults=1&currencyCode=EUR'}}, 'data': [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2023-08-31', 'lastTicketingDateTime': '2023-08-31', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT11H40M', 'segments': [{'departure': {'iataCode': 'BER', 'at': '2023-09-30T10:45:00'}, 'arrival': {'iataCode': 'AMS', 'at': '2023-09-30T12:10:00'}, 'carrierCode': 'KL', 'number': '1820', 'aircraft': {'code': 'E90'}, 'duration': 'PT1H25M', 'id': '210', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'AMS', 'at': '2023-09-30T14:10:00'}, 'arrival': {'iataCode': 'DEL', 'terminal': '3', 'at': '2023-10-01T01:55:00'}, 'carrierCode': 'KL', 'number': '871', 'aircraft': {'code': '781'}, 'operating': {'

Convert that sample nested JSON data into separate DataFrames

In [31]:
# Flight offers DataFrame
for r in response: 
    offers_df = pd.DataFrame(r['data'])
    # Itineraries DataFrame
    itineraries_df = pd.json_normalize(r['data'], record_path='itineraries', meta=['id', 'source'])
    # # Traveler pricing DataFrame
    traveler_pricing_df = pd.json_normalize(r['data'], record_path=['travelerPricings'], meta=['id', 'source'])


In [32]:
exploded_df.head()

,duration,segments,id,source
0,PT12H10M,"{'departure': {'iataCode': 'FRA', 'terminal': ...",1,GDS
0,PT12H10M,"{'departure': {'iataCode': 'DEL', 'terminal': ...",1,GDS
1,PT14H5M,"{'departure': {'iataCode': 'FRA', 'terminal': ...",2,GDS
1,PT14H5M,"{'departure': {'iataCode': 'DEL', 'terminal': ...",2,GDS
2,PT18H35M,"{'departure': {'iataCode': 'FRA', 'terminal': ...",3,GDS


Treating the itineraries df

In [27]:
# Explode the 'segment' column to create separate rows for each dictionary
exploded_df = itineraries_df.explode('segments')

# Apply json_normalize to the 'segment' column to split dictionaries into separate columns
# normalized_df = json_normalize(exploded_df['segments'], meta=['id', 'source']).add_suffix('_segments')
normalized_df = json_normalize(exploded_df['segments'], meta=['id', 'source']).add_suffix('_segments')

# # Join the exploded and normalized DataFrames
# # final_df = exploded_df.drop('segments', axis=1).join(normalized_df)
# final_df = exploded_df.drop('segments', axis=1).join(normalized_df)

# # Drop duplicates based on all columns to deduplicate the DataFrame
# deduplicated_df = final_df.drop_duplicates()

# # If you want to reset the index of the deduplicated DataFrame
# deduplicated_df.reset_index(drop=True, inplace=True)

# #dropping the second id column
# deduplicated_df=deduplicated_df.drop(columns=['id_segments'])

# # Remove the added '_segment' suffix from column names
# deduplicated_df.columns = deduplicated_df.columns.str.replace('_segments$', '', regex=True)

# #replace . in column anmes by underscore
# deduplicated_df.columns = deduplicated_df.columns.str.replace('.', '_', regex=False)

# #Assigning to itineraries df
# itineraries_df = deduplicated_df

# #drop unwanted columns 
# itineraries_df=itineraries_df.drop(columns=['blacklistedInEU','departure_terminal','arrival_terminal'])

In [118]:
final_df.head()

,duration,id,source,carrierCode_segments,number_segments,duration_segments,id_segments,numberOfStops_segments,blacklistedInEU_segments,departure.iataCode_segments,departure.terminal_segments,departure.at_segments,arrival.iataCode_segments,arrival.terminal_segments,arrival.at_segments,aircraft.code_segments,operating.carrierCode_segments
0,PT12H5M,1,GDS,AI,120,PT7H40M,23,0,False,FRA,1,2023-10-28T21:15:00,DEL,3,2023-10-29T08:25:00,788,AI
0,PT12H5M,1,GDS,AI,120,PT7H40M,23,0,False,FRA,1,2023-10-28T21:15:00,DEL,3,2023-10-29T08:25:00,788,AI
1,PT14H,2,GDS,AI,429,PT2H55M,24,0,False,DEL,3,2023-10-29T09:55:00,MAA,1,2023-10-29T12:50:00,321,AI
1,PT14H,2,GDS,AI,429,PT2H55M,24,0,False,DEL,3,2023-10-29T09:55:00,MAA,1,2023-10-29T12:50:00,321,AI
2,PT18H30M,3,GDS,AI,120,PT7H40M,5,0,False,FRA,1,2023-10-28T21:15:00,DEL,3,2023-10-29T08:25:00,788,AI


Finding out which columns in a dataframe are lists 

In [64]:
# Assuming deduplicated_df is your DataFrame
# list_columns = []

# for column in final_df.columns:
#     if final_df[column].apply(lambda x: isinstance(x, list)).any():
#         list_columns.append(column)

# print("Columns with lists:", list_columns)

Columns with lists: ['stops_segments']


Treating traveler_pricing_df

In [70]:
# Explode the 'segment' column to create separate rows for each dictionary
exploded_df = traveler_pricing_df.explode('fareDetailsBySegment')

# Apply json_normalize to the 'segment' column to split dictionaries into separate columns
normalized_df = json_normalize(exploded_df['fareDetailsBySegment']).add_suffix('_fare')

# Join the exploded and normalized DataFrames
final_df = exploded_df.drop('fareDetailsBySegment', axis=1).join(normalized_df)

# Drop duplicates based on all columns to deduplicate the DataFrame
deduplicated_df = final_df.drop_duplicates()

# # If you want to reset the index of the deduplicated DataFrame
deduplicated_df.reset_index(drop=True, inplace=True)
# deduplicated_df=deduplicated_df.drop(columns=['id_segments'])

# # Remove the added '_segment' suffix from column names
deduplicated_df.columns = deduplicated_df.columns.str.replace('_fare$', '', regex=True)

# #replace . in column anmes by underscore
deduplicated_df.columns = deduplicated_df.columns.str.replace('.', '_', regex=False)

# #Assigning to itineraries df
traveler_pricing_df = deduplicated_df

# #drop unwanted columns 
traveler_pricing_df=traveler_pricing_df.drop(columns=['fareBasis','class','brandedFare'])


---------------------------------------------------

In [71]:
now = datetime.now()
logger.info(f'Current date: {now}')

INFO-__main__:2-Current date: 2023-08-26 22:28:14.274441


Dropping unwanted columns

In [72]:
itineraries_df['incremental_day']=now
traveler_pricing_df['incremental_day']=now

In [73]:
credentials=config.pg_credentials
db = DBManager(credentials=credentials)
db.create_table_from_df(df=itineraries_df,table_name='itineraries')
db.create_table_from_df(df=traveler_pricing_df,table_name='traveler_pricing')

INFO-db_manager:31-CONNECTED
INFO-db_manager:66-Dumped to DB
INFO-db_manager:66-Dumped to DB
